データ読み込み

In [6]:
from sklearn.datasets import load_wine#サンプルデータセットのライブラリ
import pandas as pd#テーブルデータや時系列データを操作するためのデータ構造と演算のライブラリ

data = load_wine()#データ読み込み
data#dataを表示

目的変数と説明変数のテーブルデータに変換


In [7]:
train_x = pd.DataFrame(data=data.data,columns=data.feature_names)#説明変数のテーブルデータに変換。データ:data,カラム:columns
train_x.head()#train_xのデータフレームの最初の5行表示

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0


In [8]:
train_y = pd.DataFrame(data=data.target,columns=['class'])#目的変数のテーブルデータに変換。データ:data,カラム:columns
train_y.head()#train_yのデータフレームの最初の5行表示

,class
0,0
1,0
2,0
3,0
4,0


In [ ]:
from sklearn.metrics import log_loss, accuracy_score#混同行列を計算するライブラリ
from sklearn.model_selection import KFold#ホールドアウト、交差検証(cross validation)するライブラリ
import lightgbm as lgb#LightGBMのライブラリ
import numpy as np#数値計算するライブラリ

# 各foldのスコアを保存する空のリスト
scores_accuracy = []
scores_logloss = []


# クロスバリデーションを行う
# 学習データを4つに分割し、うち1つをバリデーションデータとすることを、バリデーションデータを変えて繰り返す
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_idx, va_idx in kf.split(train_x):
    # 学習データを学習データとバリデーションデータに分ける
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    # モデルの学習を行う
    model = lgb.LGBMClassifier() # モデルのインスタンスの作成
    model.fit(tr_x, tr_y) # モデルの学習

    # テストデータの予測クラス (予測クラス(0 or 1)を返す)
    y_pred = model.predict(va_x)
    # テストデータのクラス予測確率 (各クラスの予測確率 [クラス0の予測確率,クラス1の予測確率] を返す)
    y_pred_prob = model.predict_proba(va_x)
    
    
    # モデル評価
    # acc : 正答率
    accuracy = accuracy_score(va_y,y_pred)
    print('accuracy :',accuracy)

    # logloss 
    logloss =  log_loss(va_y,y_pred_prob) # 引数 : log_loss(正解クラス,[クラス0の予測確率,クラス1の予測確率])
    print('logloss :', logloss)


    # そのfoldのスコアを保存する
    scores_logloss.append(logloss)
    scores_accuracy.append(accuracy)

# 各foldのスコアの平均を出力する
logloss = np.mean(scores_logloss)
accuracy = np.mean(scores_accuracy)
print("===平均スコア===")
print(f'logloss: {logloss:.4f}, accuracy: {accuracy:.4f}')
# logloss: 0.4270, accuracy: 0.8148（本書中の数値と異なる可能性があります）